## notebook setup and configuration

In [1]:
import numpy as np
import matplotlib.pyplot as plt
import petsc4py
from petsc4py import PETSc

# basic assembly and solution of linear system

In [2]:
# problem size
n = 100

## creation of matrix `A`

Create matrix.  When using MatCreate(), the matrix format can be specified at runtime.

Performance tuning note:  For problems of substantial size, preallocation of matrix memory is crucial for attaining good performance. See the matrix chapter of the users manual for details.

In [3]:
# create the tridiagonal matrix A
A = PETSc.Mat().createAIJ([n, n], nnz=3)

Assemble the matrix


In [4]:
# first row
A.setValue(row=0, col=0, value=  2.0)
A.setValue(row=0, col=1, value= -1.0)

# inner rows
for i in range(1, n-1):
    A.setValue(row=i, col=i-1, value= -1. )
    A.setValue(row=i, col=i  , value=  2.0)
    A.setValue(row=i, col=i+1, value= -1. )
    
# last row
A.setValue(row=n-1, col=n-2, value= -1.0)
A.setValue(row=n-1, col=n-1, value=  2.0)

In [5]:
# assembly complete, make matrix usable
A.assemblyBegin() 
A.assemblyEnd()

## creation of solution vector `x` and right hand side vector`b`

$Ax = b$, with exact solution $u = \mathbf{1}^T$; we evaluate $b = A*u$

In [6]:
# create solution vector, exact solution, and right hand side
x = PETSc.Vec().createSeq(n)
u = PETSc.Vec().createSeq(n)
b = PETSc.Vec().createSeq(n)

In [7]:
# set all entries of u to 1.0
u.set(1.)

In [8]:
A.mult(u, b)

## creation of linear `KSP` solver and setting solver options

In [9]:
# create the solver
ksp = PETSc.KSP().create()
ksp.setOperators(A)

## solving the linear system

In [10]:
# call the solver
ksp.solve(b, x)

In [11]:
# check number of iterations
ksp.getIterationNumber()

1

We can multiply a vector by a scalar using the `petsc4py` analogy to the `PETSc` `VecAXPY` method, which computes $x= \alpha x + x$. Since the exact solution is $\mathbf{1}^T$, the result of $x - 1$ should be $\mathbf{0}^T$. We can check the vector norm of the result to be sure.

In [12]:
# check the error
α = -1.0
x.axpy(α, u)
x.norm(norm_type=PETSc.NormType.NORM_2)

3.2595021122271574e-14

## converting `petsc4py` objects to `NumPy` objects

In [13]:
# get u as a numpy array
u_numpy = u.getArray()
u_numpy

array([1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.,
       1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.,
       1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.,
       1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.,
       1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.,
       1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.])

## cleanup of objects in memory

In [14]:
del A, ksp

# Alternative serial constructions 

In the following section, we consider different methods of serial assembly of `A`. We define a convenience function to check that each `A` we assemble returns the correct solution as above. Note that the function can see the globally defined variables `x,b`.

In [18]:
def test_A_operator(A):
    # solve the system and check that we recover the same solution
    ksp = PETSc.KSP().create()
    ksp.setOperators(A)
    ksp.solve(b, x)
    assert np.allclose(x.getArray(), np.ones(n))
    print('test passed. A consistent.')

## using `setValues`

Instead of setting each value individually, we can set the values for each row at once by calling the `setValues` method of `A`.

In [19]:
# create the tridiagonal matrix A
A = PETSc.Mat().createAIJ([n, n], nnz=3)

# first row
A.setValues(rows=[0], cols=[0, 1], values=[2.0, -1.0])

# inner rows
for i in range(1, n-1):
    I, J = np.array([i], dtype=np.int32), np.array([i-1, i, i+1], dtype=np.int32)
    VALS = np.array([-1., 2.0, -1.])
    A.setValues(rows=I, cols=J, values=VALS)
    
# last row
A.setValues(rows=[n-1], cols=[n-1, n-2], values=[-1.0, 2.0])

# assembly complete, make matrix usable
A.assemblyBegin() 
A.assemblyEnd()

In [20]:
test_A_operator(A)

test passed. A consistent.
